In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import epiweeks

In [2]:
# Initialize Browser
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [C:\Users\Drew\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [3]:
# Visit gender data source url
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTLO0Kf-G8sP1If_Fhj5GapXHSchZ3OwLJmGq6fF2h9-gE8rHrNxyhD_nm4Oa4YicmhtZ3MLcBoQxbH/pubhtml?wmode=opaque#"
browser.visit(url)

time.sleep(1)

In [4]:
# Initialize column list
columns = ['location','date','cases_total','male_cases',
           'female_cases','percent_male_cases',
           'percent_female_cases','male_case_rate',
           'female_case_rate','deaths_total','male_deaths',
           'female_deaths','percent_male_deaths',
           'percent_female_deaths','male_death_rate',
           'female_death_rate']

In [27]:
# Scrape site and return a dataframe with the displayed week's data
def scrape_weekly_data():
    
    # Scrap into soup
    html = browser.html
    soup = bs(html, "html.parser")
    
    displayed_table = soup.find('div', style='position: relative;')
    
    # Initialize week's dataframe
    week_df = pd.DataFrame(columns=columns)
    
    # Traverse rows 5-57 containing data cells
    for row_index in range(5,58):
        row = displayed_table.find('div', text=row_index).parent.parent
        
        # Get data from relevant cells
        cells = row.find_all('td')[0:16]
        
        data = []
        
        for cell in cells:
            data.append(cell.get_text())
        
        # Add row to week table list
        week_df.loc[len(week_df)] = data
    
    return week_df

In [28]:
week_buttons = browser.find_by_tag('ul').first.find_by_tag('li')

# Create blank csv file with headers
init_df = pd.DataFrame(columns=columns)
init_df.to_csv('gender_data_raw.csv', index=False)

for button in week_buttons:
    button.click()
    week_df = scrape_weekly_data()
    week_df.to_csv('gender_data_raw.csv', mode='a', index=False, header=False)

In [29]:
result_df = pd.read_csv('gender_data_raw.csv')
result_df

,location,date,cases_total,male_cases,female_cases,percent_male_cases,percent_female_cases,male_case_rate,female_case_rate,deaths_total,male_deaths,female_deaths,percent_male_deaths,percent_female_deaths,male_death_rate,female_death_rate
0,Alabama,4/13,"4,572",1876,2615,41.03,57.20,79.63,104.24,96,56,40,58.80,41.20,2.36,1.58
1,Alaska,4/13,272,134,138,49.30,50.70,34.78,39.07,8,NaN,NaN,NaN,NaN,0.00,0.00
2,Arizona,4/13,3539,1663,1876,47.00,53.00,48.16,53.69,169,100,69,59.00,41.00,2.89,1.98
3,Arkansas,4/13,1280,552,728,43.10,56.90,37.57,47.84,27,NaN,NaN,NaN,NaN,0.00,0.00
4,California,4/13,"21,794",10926,10571,50.13,48.50,56.16,53.67,651,NaN,NaN,NaN,NaN,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4447,Virginia,11/20,953460,451359,489277,47.34,51.32,"10,906.15","11,444.54",14492,7600,6882,52.44,47.49,183.64,160.97
4448,Washington,11/20,762118,368103,373590,48.30,49.02,"10,097.01","10,239.08",9110,5041,3924,55.34,43.07,138.29,107.54
4449,West Virginia,11/20,288967,135934,149995,47.04,51.91,"15,033.69","16,218.17",4726,NaN,NaN,NaN,NaN,NaN,NaN
4450,Wisconsin,11/20,844598,407479,435460,48.25,51.56,"14,186.13","14,984.77",8848,NaN,NaN,NaN,NaN,0.00,0.00


In [30]:
result_df[result_df['location'] == 'Alabama']

,location,date,cases_total,male_cases,female_cases,percent_male_cases,percent_female_cases,male_case_rate,female_case_rate,deaths_total,male_deaths,female_deaths,percent_male_deaths,percent_female_deaths,male_death_rate,female_death_rate
0,Alabama,4/13,"4,572",1876,2615,41.03,57.20,79.63,104.24,96,56,40,58.80,41.20,2.36,1.58
53,Alabama,4/27,"6,421",2650,3706,41.27,57.72,112.49,147.72,219,129,90,58.70,41.30,5.46,3.61
106,Alabama,5/4,"7,888",3202,4561,40.59,57.82,135.91,181.79,290,162,128,55.90,44.10,6.88,5.10
159,Alabama,5/11,"10,468",4311,6041,41.18,57.71,182.98,240.79,440,244,196,55.50,44.50,10.37,7.80
212,Alabama,5/18,"12,042",4977,6964,41.33,57.83,211.26,277.57,488,269,219,55.20,44.80,11.43,8.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,Alabama,10/23,"819,597",NaN,NaN,NaN,NaN,NaN,NaN,"15,406",NaN,NaN,NaN,NaN,NaN,NaN
4240,Alabama,10/30,"831,653",NaN,NaN,NaN,NaN,NaN,NaN,"15,573",NaN,NaN,NaN,NaN,NaN,NaN
4293,Alabama,11/6,"835,740",NaN,NaN,NaN,NaN,NaN,NaN,"15,834",NaN,NaN,NaN,NaN,NaN,NaN
4346,Alabama,11/13,"839,436",NaN,NaN,NaN,NaN,NaN,NaN,"15,981",NaN,NaN,NaN,NaN,NaN,NaN
